# Segmenting and Clustering Neighbourhoods in Toronto

* Start with importing the necessary python libraries

In [1]:
import pandas as pd
import numpy  as np
from bs4 import BeautifulSoup
import requests

#### Get the table with postal codes for Toronto from the internet
* Get the contents of a wikipedia page with Toronto postal codes including their matching boroughs and neighbourhods
* Show the response code = 200 = return OK

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

#### Get the data from the HTML contents
* Parse the page contents to get all of the HTML tag elements
* To get an idea of what the contents look like you can use
```python
    dom.prettify
```
* We have found out that the contents are in the only table in the HTML contents
* Just to make sure add the tables class to the find criteria
* Then get all the TD elements contaning the data we need

In [3]:
# Parse the page contents to get all of the HTML tag elements
dom = BeautifulSoup(page.content, 'html.parser')
# Get the table containing the data records
tab = dom.find('table',class_='wikitable sortable')
# within the table get all the TD's containg the data column values
tds = tab.find_all('td')

#### We are now ready to parse the data en fill the dataframe
* Loop through all the TD elements to get the data
* Every first element is the postal code (counter mod 3 == 0)
* Every second element is the borough (counter mod 3 == 1)
* Every third element is the neighbourhood (counter mod 3 == 2)
* Append the postal codes, boroughs and neighbourhoods found each to a separate list
* Use the lists to create a dataframe to work with

In [4]:
# get the postal code, borough and neighbourhood from
# the table's TD elements text
# 1st TD = postal code
# 2nd TD = borough
# 3rd TD = neighbourhood
postal_codes   = []
boroughs       = []
neighbourhoods = []
for i,el in enumerate(tds):
    # get the elements text and remove any leading or trailing spaces, new lines
    txt = el.get_text().strip()
    if i % 3 == 0:
        postal_codes.append(txt)
    elif i % 3 == 1:
        boroughs.append(txt)
    elif i % 3 == 2:
        neighbourhoods.append(txt)
# creaate the dataframe with postal code, borough and neighbourhood
df = pd.DataFrame({
    "postal_code" : postal_codes,
    "borough" : boroughs,
    "neighbourhood" : neighbourhoods
})
# list the rows and columns
print(df.shape)
# get an idea of what has been created
df.head(6)

(289, 3)


,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


#### Remove any lines where the borough is equal to 'Not assigned'
* print the number of rows before and after removing as an additional check

In [5]:
# clean up the table
# remove any lines with borough = Not assigned
print('Shape before removing "Not asigned"',df.shape)
df = df[df['borough'] != 'Not assigned']
print('Shape after removing "Not asigned"',df.shape)

Shape before removing "Not asigned" (289, 3)
Shape after removing "Not asigned" (212, 3)


* Find out if there are any rows which have a neighbourhood = 'Not assigned'
* Then fix this by assigning the borough name to the neighbourhood

In [6]:
# check which rows have neighbourhood = 'Not assigned'
df[df['neighbourhood'] == 'Not assigned']

,postal_code,borough,neighbourhood
8,M7A,Queen's Park,Not assigned


In [7]:
# assign the borough to the neighbourhood id the neighbourhood is 'Not assigned'
df['neighbourhood'] = np.where(df['neighbourhood']=='Not assigned', df['borough'], df['neighbourhood'])

#### Take care of rows with duplicate postal codes
1. Sort the dataframe by postal code and neighbourhood
2. Reset the index column afterwards
3. Create one dataframe including all duplicate rows based on the postal code column
  * We need this dataframe to collect the neighbourhoods by postal code
4. Create a second dataframe including the first appearance of duplicat rows based on the postal code column
  * We will use this later on to merge with the neighbourhoods collection by postal code
5. Create a third datafrme including rows with only unique postal codes
  * We will use this later on to merge into to one final dataframe to be used

In [8]:
df.sort_values(by=['postal_code','neighbourhood'],inplace=True)
df.reset_index(drop=True,inplace=True)
# which rows have a duplicate postal code?
# one dataframe with all rows that have duplicate postal codes
df_dup     = df[df.duplicated(subset='postal_code',keep=False)]
# one dataframe with only the first row that has a duplicate postal code
df_dup_1st = df[df.duplicated(subset='postal_code',keep='first')]
# one dataframe with all rows with unique postal codes
df_uni     = df[~df.duplicated(subset='postal_code',keep=False)]

#### Create a dictionary to merge all neighbourhoods under one postal code
* Loop through all the rows and get the postal code and neighbourhood
* Append the neighbourhood as a value to the postal code key record

In [9]:
pcd_dict = dict()
for i in range(len(df_dup)):
    nbh = df_dup['neighbourhood'].values[i] 
    pcd = df_dup['postal_code'].values[i]
    # add or append to the dictionary key values
    pcd_dict.setdefault(pcd, []).append(nbh)

* do some checks on row counts of each dataframe

In [10]:
# some count checks just to make sure
print('Row count orignal dataframe: ',len(df))
print('Row count dataframe with unique postal codes: ',len(df_uni))
print('Row count dataframe with all rows with duplicate postal codes: ',len(df_dup))
print('Row count with the first row of rows with duplicate postal codes: ',len(df_dup_1st))

Row count orignal dataframe:  212
Row count dataframe with unique postal codes:  45
Row count dataframe with all rows with duplicate postal codes:  167
Row count with the first row of rows with duplicate postal codes:  109


#### Merge the dataframe with duplicate rows containing the first record with the dictionary
* Loop at the rows of the dataframe and get the postal code for each row
* Set the neighbourhood column to the dictionary value (= list of neighbourhoods wit the same postal code)
  * This is a string with all the neighbourhood names joined by a ', '

In [11]:
# replace the neighbourhood column with the dictionary values
for i in range(len(df_dup_1st)):
    pcd = df_dup_1st['postal_code'].values[i]
    df_dup_1st['neighbourhood'].values[i] = ', '.join(pcd_dict[pcd])

#### Merge the dataframe with the unique postal codes with the dataframe containing the duplicate rows

In [12]:
df_new = pd.concat([df_uni, df_dup_1st],ignore_index=True)
df_new.shape

(154, 3)

#### Remove any rows with duplicate records

In [13]:
# remove duplicate lines based on the postal code column
df_new.drop_duplicates(subset='postal_code',inplace=True)

* The final dataframe should only have the number of row as there are unique postal codes

In [14]:
# do a check on the number of unique postal codes compared with the number of rows
# in the dataframe
print('Numer of unique postal codes: ',len(df_new['postal_code'].unique()))
print('Rows and columns of the dataframe: ', df_new.shape)

Numer of unique postal codes:  103
Rows and columns of the dataframe:  (103, 3)


In [15]:
df_new.tail()

,postal_code,borough,neighbourhood
136,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
140,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
143,M9M,North York,"Emery, Humberlea"
144,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
147,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [16]:
df_new.shape

(103, 3)